In [1]:
import pandas as pd

RECORD_DATA_FP = 'data/tmp/6.24号以后reversible_lanes_data.xls'

record_df = pd.read_excel(RECORD_DATA_FP)
print(record_df.loc[0, 'json_data'])

{"duration": 1200, "laneData": [{"flow": 12, "laneId": 16, "queueNum": 0, "queueLength": 1}, {"flow": 81, "laneId": 17, "queueNum": 1, "queueLength": 3}, {"flow": 72, "laneId": 18, "queueNum": 1, "queueLength": 4}, {"flow": 72, "laneId": 19, "queueNum": 0, "queueLength": 0}, {"flow": 63, "laneId": 30, "queueNum": 0, "queueLength": 0}, {"flow": 33, "laneId": 31, "queueNum": 0, "queueLength": 0}, {"flow": 69, "laneId": 32, "queueNum": 1, "queueLength": 3}, {"flow": 96, "laneId": 33, "queueNum": 0, "queueLength": 0}], "timestamp": 1687622403}


In [2]:
import json

LANE_IDS = {16, 17, 18, 19, 30, 31, 32, 33}

def lane_flow_col_ids():
    return  [f'lane{lane_id}Flow' for lane_id in LANE_IDS]

def _filter_road_status(info: dict):
    if not 'laneData' in info:
        return False

    all_lanes = {item['laneId'] for item in info['laneData']}
    if not LANE_IDS == all_lanes:
        return False
    return True

def _unpack_data(series: pd.Series):
    info =  series['json_data']
    new_series = pd.Series({'timestamp': info['timestamp'], 'duration': info['duration']})
    for lane_data in info['laneData']:
        lane_id = lane_data['laneId']
        flow = lane_data['flow']
        new_series[f'lane{lane_id}Flow'] = flow
    return new_series

    

record_df['json_data'] = record_df['json_data'].apply(json.loads)
traffic_state_data_index = record_df['json_data'].map(_filter_road_status)
related_df = record_df.loc[traffic_state_data_index, :]

tf_df = related_df.apply(_unpack_data, axis=1)
print(tf_df)

       timestamp  duration  lane16Flow  lane17Flow  lane18Flow  lane19Flow  \
0     1687622403      1200          12          81          72          72   
1     1687623603      1200           6          66          51          45   
2     1687624803      1200          18          45          45          24   
3     1687626003      1200           9          51          33          78   
4     1687627203      1200           3          57          45          45   
...          ...       ...         ...         ...         ...         ...   
1079  1688869189      1200          78         180         108         180   
1080  1688870389      1200          93         219         162         174   
1081  1688871589      1200          78         192         150         150   
1082  1688872789      1200         111         204          78         201   
1083  1688873989      1200          78         189         108         189   

      lane30Flow  lane31Flow  lane32Flow  lane33Flow  
0       

In [3]:
## accumulate data
import time
from collections import defaultdict
UPDATE_INTERVAL = 20  # 更新间隔时间

accumulate_weekday_flow = defaultdict(dict)
accumulate_weekend_flow = defaultdict(dict)

def calculate_avg_flow(flow_data):
    for period_decimal, _flow in flow_data.items():
        for lane_id in _flow.keys():
            all_flow = _flow[lane_id]
            _flow[lane_id] = sum(all_flow) / len(all_flow)
    return dict(sorted(flow_data.items(), key=lambda x: x[0]))

for index, row in tf_df.iterrows():
    time_struct = time.localtime(row['timestamp'])
    time_hour_decimal = (time_struct.tm_hour * 60 + time_struct.tm_min) // UPDATE_INTERVAL    
    data_container = accumulate_weekend_flow if time_struct.tm_wday in (5, 6) else accumulate_weekday_flow
    for lane_id, lane_col_name in zip(LANE_IDS, lane_flow_col_ids()):
        lane_period_data = data_container[time_hour_decimal].setdefault(lane_id, [])
        lane_period_data.append(row[lane_col_name])

weekday_flow = calculate_avg_flow(accumulate_weekday_flow)
weekend_flow = calculate_avg_flow(accumulate_weekend_flow)

In [4]:
from lib.SPAT import Movement, Direction, Turn
from lib.state import TurnDemand, DayLanePlan, PlanDuration
from src.lane_change import StaticIntersectionController, VarianceLane, LaneAllocation, VMS

STRAIGHT_SAT_RATE = 1600
TURN_SAT_RATE = 900
LEFT_SAT_RATE = 1300
RIGHT_SAT_RATE = 1400

LANE_MOVEMENT_MAPPING = {
    16: Movement(Direction.EAST, Turn.STRAIGHT),
    17: Movement(Direction.EAST, Turn.STRAIGHT),
    18: Movement(Direction.EAST, Turn.STRAIGHT),
    19: Movement(Direction.EAST, Turn.RIGHT_TURN),
    30: Movement(Direction.WEST, Turn.LEFT),
    31: Movement(Direction.WEST, Turn.STRAIGHT),
    32: Movement(Direction.WEST, Turn.STRAIGHT),
    33: Movement(Direction.WEST, Turn.STRAIGHT)
}

# 只有东进口变
PEAK_LANE_MOVEMENT_MAPPING1 = {
    16: Movement(Direction.EAST, Turn.STRAIGHT),
    17: Movement(Direction.EAST, Turn.STRAIGHT),
    18: Movement(Direction.EAST, Turn.TURN),
    19: Movement(Direction.EAST, Turn.RIGHT),
    30: Movement(Direction.WEST, Turn.LEFT),
    31: Movement(Direction.WEST, Turn.STRAIGHT),
    32: Movement(Direction.WEST, Turn.STRAIGHT),
    33: Movement(Direction.WEST, Turn.STRAIGHT)
}

# 两者都变
PEAK_LANE_MOVEMENT_MAPPING2 = {
    16: Movement(Direction.EAST, Turn.STRAIGHT),
    17: Movement(Direction.EAST, Turn.STRAIGHT),
    18: Movement(Direction.EAST, Turn.TURN),
    19: Movement(Direction.EAST, Turn.RIGHT),
    30: Movement(Direction.WEST, Turn.LEFT),
    31: Movement(Direction.WEST, Turn.LEFT),
    32: Movement(Direction.WEST, Turn.STRAIGHT),
    33: Movement(Direction.WEST, Turn.STRAIGHT)
}

# 只有西进口改变
PEAK_LANE_MOVEMENT_MAPPING3 = {
    16: Movement(Direction.EAST, Turn.STRAIGHT),
    17: Movement(Direction.EAST, Turn.STRAIGHT),
    18: Movement(Direction.EAST, Turn.STRAIGHT),
    19: Movement(Direction.EAST, Turn.RIGHT_TURN),
    30: Movement(Direction.WEST, Turn.LEFT),
    31: Movement(Direction.WEST, Turn.LEFT),
    32: Movement(Direction.WEST, Turn.STRAIGHT),
    33: Movement(Direction.WEST, Turn.STRAIGHT)
}

SAT_THRESHOLD = {
    Turn.LEFT: 0.85,
    Turn.STRAIGHT: 0.75,
    Turn.RIGHT: 0.85,
    Turn.TURN: 0.75
}

east_basic_lane_allocation = {Turn.STRAIGHT: 2, Turn.TURN: 0, Turn.RIGHT: 0}
east_flexible_lane_allocation = [
    {
        Turn.STRAIGHT: 1,
        Turn.TURN: 0.5,
        Turn.RIGHT: 0.5
    },
    {
        Turn.STRAIGHT: 0,
        Turn.TURN: 1,
        Turn.RIGHT: 1
    }
]
east_lane_allocation = LaneAllocation(east_basic_lane_allocation, east_flexible_lane_allocation)

west_basic_lane_allocation = {Turn.STRAIGHT: 2, Turn.LEFT: 1}
west_flexible_lane_allocation = [
    {
        Turn.STRAIGHT: 1,
        Turn.LEFT: 0,
    },
    {
        Turn.STRAIGHT: 0,
        Turn.LEFT: 1
    }
]
west_lane_allocation = LaneAllocation(west_basic_lane_allocation, west_flexible_lane_allocation)

cycle_length = 200
east_green_spilt = {Turn.STRAIGHT: 88 / cycle_length, Turn.TURN: 58 / cycle_length,
                    Turn.RIGHT: (1., 1 - (1 - 58 / cycle_length) * 0.3)}
west_green_spilt = {Turn.STRAIGHT: 88 / cycle_length, Turn.LEFT: 58 / cycle_length}


def east_vms_state_output_func(turn: Turn):
    vms_3 = {'vmsId': 3, 'laneId': 19, 'direction': 2}  # 最右侧
    vms_4 = {'vmsId': 4, 'laneId': 18, 'direction': 2}  # 次右侧

    if turn is Turn.STRAIGHT:
        vms_3['movement'] = 8
        vms_4['movement'] = 1
    elif turn is Turn.RIGHT:
        vms_3['movement'] = 3
        vms_4['movement'] = 4
    else:
        raise NotImplementedError('invalid priority turn for east entry link')
    return [vms_3, vms_4]


def west_vms_state_output_func(turn: Turn):
    vms_1 = {'vmsId': 1, 'laneId': 31, 'direction': 4}  # 下游
    vms_2 = {'vmsId': 2, 'laneId': 31, 'direction': 4}  # 上游

    if turn is Turn.STRAIGHT:
        vms_1['movement'] = 1
        vms_2['movement'] = 1
    elif turn is Turn.LEFT:
        vms_1['movement'] = 2
        vms_2['movement'] = 2
    else:
        raise NotImplementedError('invalid priority turn for west entry link')
    return [vms_1, vms_2]


east_vms = VMS(Turn.STRAIGHT, Turn.STRAIGHT, Turn.RIGHT, entity_state_func=east_vms_state_output_func)
# east_vms1 = VMS(Turn.STRAIGHT, Turn.STRAIGHT, Turn.TURN)
# east_vms2 = VMS(Turn.RIGHT_TURN, Turn.RIGHT_TURN, Turn.RIGHT)
west_vms = VMS(Turn.STRAIGHT, Turn.STRAIGHT, Turn.LEFT, entity_state_func=west_vms_state_output_func)

TRUCK_RATE = 0.2
PASSENGER_RATE = 1 - TRUCK_RATE
VEH_TYPE_FACTOR = (PASSENGER_RATE + 0.5 * TRUCK_RATE)
modified_straight_rate = STRAIGHT_SAT_RATE * VEH_TYPE_FACTOR
modified_left_rate = LEFT_SAT_RATE * VEH_TYPE_FACTOR
modified_turn_rate = TURN_SAT_RATE * VEH_TYPE_FACTOR
modified_right_rate = RIGHT_SAT_RATE * VEH_TYPE_FACTOR
east_demand = [TurnDemand(Turn.STRAIGHT, modified_straight_rate), TurnDemand(Turn.TURN, modified_turn_rate),
                TurnDemand(Turn.RIGHT, modified_right_rate)]
west_demand = [TurnDemand(Turn.STRAIGHT, modified_straight_rate), TurnDemand(Turn.LEFT, modified_left_rate)]

v_lane_east = VarianceLane(Direction.EAST, east_vms, SAT_THRESHOLD, east_demand, east_lane_allocation,
                            east_green_spilt)
v_lane_west = VarianceLane(Direction.WEST, west_vms, SAT_THRESHOLD, west_demand, west_lane_allocation,
                            west_green_spilt)

# controller = StaticIntersectionController(variance_lanes=[v_lane_east, v_lane_west],
#                                           lane_movement_mapping=LANE_MOVEMENT_MAPPING,
#                                           update_interval_sec=1800,
#                                           peak_lane_movement_mapping=PEAK_LANE_MOVEMENT_MAPPING,
#                                           peak_hour_range=(17, 20))
#
# tf_data_path = 'data/TrafficFlow_Logs3.log'
# for stat in read_file(tf_data_path):
#     controller.update_from_traffic_flow(stat[0])

peak1_pd = PlanDuration(PEAK_LANE_MOVEMENT_MAPPING3, 8, 10)
peak2_pd = PlanDuration(PEAK_LANE_MOVEMENT_MAPPING3, 16, 16.5)
peak3_pd = PlanDuration(PEAK_LANE_MOVEMENT_MAPPING2, 16.5, 19)
peak4_pd = PlanDuration(PEAK_LANE_MOVEMENT_MAPPING1, 19, 20)

normal1_pd = PlanDuration(LANE_MOVEMENT_MAPPING, 0, 8)
normal2_pd = PlanDuration(LANE_MOVEMENT_MAPPING, 10, 16)
normal3_pd = PlanDuration(LANE_MOVEMENT_MAPPING, 20, 24)
day_lane_plan = DayLanePlan([peak1_pd, peak2_pd, peak3_pd, peak4_pd, normal1_pd, normal2_pd, normal3_pd])

controller = StaticIntersectionController(variance_lanes=[v_lane_east, v_lane_west],
                                          lane_movement_mapping=LANE_MOVEMENT_MAPPING,
                                          update_interval_sec=1200,  # 1800
                                          history_lane_movement_mapping=day_lane_plan)


工作日方案

In [5]:
for period, lane_flow in weekday_flow.items():
    hour = period * (UPDATE_INTERVAL / 60)
    print(hour)
    controller.update_from_complete_data(lane_flow, hour)

0.0
EAST 0.07922979797979798 0.12833545108005084 -0.04910565310025286
WEST 0.0845959595959596 0.24933687002652521 -0.1647409104305656
0.3333333333333333
EAST 0.07844065656565656 0.1228293096145701 -0.04438865304891354
WEST 0.07922979797979798 0.2732095490716181 -0.1939797510918201
0.6666666666666666
EAST 0.07007575757575757 0.12791190173655229 -0.05783614416079472
WEST 0.08285984848484848 0.24845269672855883 -0.16559284824371034
1.0
EAST 0.0634469696969697 0.09783989834815755 -0.03439292865118786
WEST 0.07402146464646465 0.21662245800176838 -0.14260099335530374
1.3333333333333333
EAST 0.05082070707070707 0.0698856416772554 -0.01906493460654833
WEST 0.056976010101010104 0.20866489832007076 -0.15168888821906065
1.6666666666666665
EAST 0.03835227272727273 0.08005082592121981 -0.04169855319394709
WEST 0.05760732323232323 0.15649867374005308 -0.09889135050772985
2.0
EAST 0.038825757575757576 0.06056755612028801 -0.021741798544530434
WEST 0.04498106060606061 0.13704686118479223 -0.0920658005

周末方案

In [6]:
controller = StaticIntersectionController(variance_lanes=[v_lane_east, v_lane_west],
                                          lane_movement_mapping=LANE_MOVEMENT_MAPPING,
                                          update_interval_sec=1200,  # 1800
                                          history_lane_movement_mapping=day_lane_plan)
for period, lane_flow in weekend_flow.items():
    hour = period * (UPDATE_INTERVAL / 60)
    print(hour)
    controller.update_from_complete_data(lane_flow, hour)

0.0
EAST 0.09343434343434345 0.13214739517153748 -0.038713051737194026
WEST 0.09311868686868686 0.25110521662245805 -0.1579865297537712
0.3333333333333333
EAST 0.08648989898989899 0.09318085556967386 -0.006690956579774865
WEST 0.08964646464646464 0.27586206896551724 -0.1862156043190526
0.6666666666666666
EAST 0.0716540404040404 0.09318085556967386 -0.021526815165633456
WEST 0.0735479797979798 0.19451812555260833 -0.12097014575462853
1.0
EAST 0.07196969696969698 0.08809826344769166 -0.016128566477994682
WEST 0.07765151515151515 0.2192749778956676 -0.14162346274415244
1.3333333333333333
EAST 0.05523989898989899 0.06437950021177467 -0.009139601221875682
WEST 0.06407828282828283 0.15384615384615388 -0.08976787101787105
1.6666666666666665
EAST 0.052083333333333336 0.0626853028377806 -0.010601969504447258
WEST 0.052083333333333336 0.18567639257294433 -0.13359305923961098
2.0
EAST 0.045138888888888895 0.06183820415078356 -0.016699315261894665
WEST 0.04671717171717172 0.15030946065428827 -0.10